In [9]:
# Read Data
f = open("data/bezdekIris.data", "r")
fData = f.readlines()
lData = []

for sIter in fData:
    if (sIter[-1] == "\n"):
        sIter = sIter[:-1]
    if (sIter == ""):
        continue
    lData.append(sIter.split(","))

print(len(lData[0]))

5
3.5


In [19]:
# Create HashMaps
lDicts = [dict() for x in range(len(lData[0]))] # array/list of dictionaries (Hashmaps)
for i in range(0, len(lData[0])):
    dFeatures = {} # Empty hashmaps
    for j in range(0, len(lData)):
        if lData[j][i] in dFeatures.keys():
            dFeatures[lData[j][i]] = dFeatures[lData[j][i]] + 1
        else:
            dFeatures[lData[j][i]] = 0
    lDicts[i] = dFeatures
print(lDicts[4])        

{'Iris-setosa': 49, 'Iris-versicolor': 49, 'Iris-virginica': 49}
